# Dremio Test

## Setting up dremio


## Imports

In [1]:
import os
import sys
import atoti as tt

In [2]:
import pandas as pd

Make sure you are running a version of atoti older than 0.8.10. This has been tested on 0.8.5

In [3]:
tt._version.VERSION

'0.9.0.20240507074627'

### Jar Input

In [4]:
dremio_driver_jar="flight-sql-jdbc-driver-15.0.0.jar"
dremio_dialect_jar="dremio-dialect-0.1.0.jar"

### SQL Setup

In [5]:
schema_name='"@admin"'
db_table_name='SampleData20240104'
sql = f"SELECT * FROM {schema_name}.{db_table_name}"

### Database setup

In [6]:
username="admin"
password="admin01!"
host_name="localhost"
port="32010"
database = "Dremio"

jdbc_legacy_url=f"jdbc:dremio:direct={host_name}:31010?user={username}&password={password}&useEncryption=false"
jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&password={password}&useEncryption=false"

jdbc_driver="org.apache.arrow.driver.jdbc.ArrowFlightJdbcDriver"
jdbc_legacy_driver="com.dremio.jdbc.Driver"
atoti_table_name="table_facts"

In [7]:
os.getcwd()

'/Users/gpadya/Documents/dremio-driver/Python Dremio Demo'

## Session

In [8]:
session = tt.Session(
    port=9092,
    user_content_storage = "./Python Dremio Demo/content",
    java_options=[
        "-Xmx10g",
        "-XX:MaxDirectMemorySize=20g",
        # "-Dorg.springframework.boot.logging.LoggingSystem=none"
    ],
    extra_jars=[f"./utilities/extra_jars/{dremio_driver_jar}",
               f"./utilities/extra_jars/{dremio_dialect_jar}"],
    logging=tt.LoggingConfig(destination="./logging/server.log")
)

# Configuration of the Data model

## Connect to Dremio and Create External DataStore

In [9]:
from atoti_directquery_jdbc import JDBCConnectionInfo

In [10]:
from atoti_directquery_jdbc import (
JDBCConnection, 
JDBCConnectionInfo,
JDBCTable,
JDBCTableOptions
)

In [11]:
jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&password={password}&useEncryption=false"

In [12]:
# jdbc_url = f"jdbc:arrow-flight-sql://{host_name}:{port}?user={username}&useEncryption=false"

In [13]:
connection_info=JDBCConnectionInfo(
    jdbc_url,
    sql_dialect_key="Dremio",
    # password=password,
    password_parameter_name='password'
    
)
external_database = session.connect_to_external_database(connection_info)

In [19]:
external_database.

## Set up DQ Tables

In [17]:
table_facts_types= {
    "AsofDate" : tt.LOCAL_DATE,
    "CUSIP" : tt.STRING,
    "SectorCode" : tt.STRING,
    "Strategy" : tt.STRING,
    "MarketValue" : tt.DOUBLE,
    "EffectiveDuration" : tt.DOUBLE,
    "OptionAdjustedSpread" : tt.DOUBLE,
    "LegalEntity" : tt.STRING,
    "PortfolioTicker" : tt.STRING,
    "OwnershipType" : tt.STRING,
    "PercentWeight" : tt.INT
}

In [18]:
table_facts_db=external_database.tables['SampleData20240104']

Py4JJavaError: An error occurred while calling o17.getTables.
: java.lang.NullPointerException
	at java.base/java.util.Objects.requireNonNull(Unknown Source)
	at java.base/java.util.ImmutableCollections$Map1.<init>(Unknown Source)
	at java.base/java.util.Map.ofEntries(Unknown Source)
	at java.base/java.util.stream.Collectors.lambda$toUnmodifiableMap$66(Unknown Source)
	at java.base/java.util.function.Function.lambda$andThen$1(Unknown Source)
	at java.base/java.util.stream.ReferencePipeline.collect(Unknown Source)
	at com.activeviam.tech.core.internal.util.CollectionUtil.mapValues(CollectionUtil.java:76)
	at io.atoti.runtime.private_.directquery.sql.ASqlDatabaseManager.toTableNames(ASqlDatabaseManager.java:390)
	at io.atoti.directquery.jdbc.private_.GenericJdbcManager.listTables(GenericJdbcManager.java:50)
	at io.atoti.runtime.private_.sql.ASqlDatabaseApi.getTables(ASqlDatabaseApi.java:83)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(Unknown Source)
	at java.base

In [ ]:
table_facts=session.add_external_table(table_facts_db)
table_facts.head(10)

In [ ]:
session.tables.schema

## Create Cube

In [ ]:
cube = session.create_cube(table_facts, name="facts", mode="no_measures")
h, l, m = cube.hierarchies, cube.levels, cube.measures

### Clean Up Hierarchies

In [ ]:
# del(h['MarketValue'],h['EffectiveDuration'], h['OptionAdjustedSpread'], h['PercentWeight'])

## Add Measures

In [ ]:
%%time
m["MarketValue"] = tt.agg.sum(table_facts["MarketValue"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["EffectiveDuration"] = tt.agg.sum(table_facts["EffectiveDuration"])
m["OptionAdjustedSpread"] = tt.agg.sum(table_facts["OptionAdjustedSpread"])
m["PercentWeight"] = tt.agg.sum(table_facts["PercentWeight"])

In [ ]:
%%time
m["MV Duration"] = tt.agg.sum_product(
    table_facts["MarketValue"],table_facts["EffectiveDuration"],table_facts["PercentWeight"])

In [ ]:
%%time
m["MV PercentWeight"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["PercentWeight"])

In [ ]:
%%time
m["MV OAS"] = tt.agg.sum_product(table_facts["MarketValue"],table_facts["OptionAdjustedSpread"],table_facts["PercentWeight"])

In [ ]:
%%time
m["Weighted Average Duration"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV Duration"]/m["MV PercentWeight"])

In [ ]:
%%time
m["Weighted Average Duration OAS"] = tt.where(m["MV PercentWeight"] == 0, 0, m["MV OAS"]/m["MV PercentWeight"])

## Link to UI

In [ ]:
session.link